In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import collections
import math

Instructions for updating:
non-resource variables are not supported in the long term


In [122]:
trainBullData = pd.read_csv("trainBullDataNorm.csv")
testBullData = pd.read_csv("testBullDataNorm.csv")
trainBullData.fillna(1, inplace=True)
testBullData.fillna(1, inplace=True)

#Basic Hyperparameter Tuning
#trainBullData = trainBullData.iloc[:10000]

#Advanced Tuning
trainBullData = trainBullData[trainBullData["Month"].str.contains("05")]


y_train = trainBullData["PRC"]
x_train = trainBullData[['VOL','Momentum', 'CAPEI', 'bm', 'evm', 'pe_inc', 'ps', 'pcf','ptpm', 'roa', 'roe', 'roce', 'aftret_invcapx', 'aftret_equity','GProf', 'equity_invcap', 'debt_invcap', 'capital_ratio', 'invt_act','rect_act', 'cash_debt', 'dltt_be', 'de_ratio', 'cash_ratio','quick_ratio', 'curr_ratio', 'rd_sale', 'ptb']]
y_train = pd.DataFrame(y_train)
x_train = pd.DataFrame(x_train)

In [123]:
x = tf.placeholder("float", shape= [None, 28])
y = tf.placeholder("float", shape= [None, 1])

weights = {
    'h1': tf.Variable(tf.random_normal([28, 32], stddev=1/28)),
    'h1Out': tf.Variable(tf.random_normal([32, 1], stddev=1/32)), 'h2': tf.Variable(tf.random_normal([32, 16], stddev=1/32)),
    'h2Out': tf.Variable(tf.random_normal([16, 1], stddev=1/16)), 'h3': tf.Variable(tf.random_normal([16, 8], stddev=1/16)),
    'h3Out': tf.Variable(tf.random_normal([8, 1], stddev=1/8)), 'h4': tf.Variable(tf.random_normal([8, 4], stddev=1/8)),
    'h4Out': tf.Variable(tf.random_normal([4, 1], stddev=1/4)), 'h5': tf.Variable(tf.random_normal([4, 2], stddev=1/4)),
    'h5Out': tf.Variable(tf.random_normal([2, 1], stddev=1/2))}
biases = {
    'b1': tf.Variable(tf.random_normal([32], stddev=1/28)), 'b2': tf.Variable(tf.random_normal([16], stddev=1/32)), 'b3': tf.Variable(tf.random_normal([8], stddev=1/16)), 'b4': tf.Variable(tf.random_normal([4], stddev=1/8)), 'b5': tf.Variable(tf.random_normal([2], stddev=1/4)), 'bOut': tf.Variable(tf.random_normal([1], stddev=1/28))}

In [124]:
def singleLayer(x, weights, biases):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layerOut = tf.matmul(layer1, weights['h1Out'])
    return layerOut
    
def doubleLayer(x, weights, biases):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.matmul(layer1, weights['h2'])
    layer2 = tf.nn.relu(layer2)
    layerOut = tf.matmul(layer2, weights['h2Out'])
    return layerOut
    
def tripleLayer(x, weights, biases):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.matmul(layer1, weights['h2'])
    layer2 = tf.nn.relu(layer2)
    layer3 = tf.matmul(layer2, weights['h3'])
    layer3 = tf.nn.relu(layer3)
    layerOut = tf.matmul(layer3, weights['h3Out'])
    return layerOut

def QuadLayer(x, weights, biases):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.matmul(layer1, weights['h2'])
    layer2 = tf.nn.relu(layer2)
    layer3 = tf.matmul(layer2, weights['h3'])
    layer3 = tf.nn.relu(layer3)
    layer4 = tf.matmul(layer3, weights['h4'])
    layer4 = tf.nn.relu(layer4)
    layerOut = tf.matmul(layer4, weights['h4Out'])
    return layerOut

def QuintLayer(x, weights, biases):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.matmul(layer1, weights['h2'])
    layer2 = tf.nn.relu(layer2)
    layer3 = tf.matmul(layer2, weights['h3'])
    layer3 = tf.nn.relu(layer3)
    layer4 = tf.matmul(layer3, weights['h4'])
    layer4 = tf.nn.relu(layer4)
    layer5 = tf.matmul(layer4, weights['h5'])
    layer5 = tf.nn.relu(layer5)
    layerOut = tf.matmul(layer5, weights['h5Out'])
    return layerOut

#Input Effect on Output
def singleLayerInputEffects(x, weightsFinal):
    x = np.array([x.astype("float32")])
    layer1 = np.matmul(x, weightsFinal['h1'])
    devRELUh1 = RELUDerivative(layer1)
    
    temp = weightsFinal["h1Out"]
    inputEffects = np.matmul(weightsFinal["h1"], np.multiply(devRELUh1.T, temp))
    return inputEffects

def doubleLayerInputEffects(x, weightsFinal):
    x = np.array([x.astype("float32")])
    layer1 = np.matmul(x, weightsFinal['h1'])
    devRELUh1 = RELUDerivative(layer1)
    
    layer1 = RELU(layer1)
    layer2 = np.matmul(layer1, weightsFinal['h2'])
    devRELUh2 = RELUDerivative(layer2)

    temp = np.matmul(weightsFinal["h2"], np.multiply(devRELUh2.T, weightsFinal["h2Out"]))
    inputEffects = np.matmul(weightsFinal["h1"], np.multiply(devRELUh1.T, temp))
    return inputEffects

def tripleLayerInputEffects(x, weightsFinal):
    x = np.array([x.astype("float32")])
    layer1 = np.matmul(x, weightsFinal['h1'])
    devRELUh1 = RELUDerivative(layer1)

    layer1 = RELU(layer1)
    layer2 = np.matmul(layer1, weightsFinal['h2'])
    devRELUh2 = RELUDerivative(layer2)

    layer2 = RELU(layer2)
    layer3 = np.matmul(layer2, weightsFinal['h3'])
    devRELUh3 = RELUDerivative(layer3)
    
    temp = np.matmul(weightsFinal["h3"], np.multiply(devRELUh3.T, weightsFinal["h3Out"]))
    temp = np.matmul(weightsFinal["h2"], np.multiply(devRELUh2.T, temp))
    inputEffects = np.matmul(weightsFinal["h1"], np.multiply(devRELUh1.T, temp))
    return inputEffects

def quadLayerInputEffects(x, weightsFinal):
    x = np.array([x.astype("float32")])
    layer1 = np.matmul(x, weightsFinal['h1'])
    devRELUh1 = RELUDerivative(layer1)

    layer1 = RELU(layer1)
    layer2 = np.matmul(layer1, weightsFinal['h2'])
    devRELUh2 = RELUDerivative(layer2)

    layer2 = RELU(layer2)
    layer3 = np.matmul(layer2, weightsFinal['h3'])
    devRELUh3 = RELUDerivative(layer3)
    
    layer3 = RELU(layer3)
    layer4 = np.matmul(layer3, weightsFinal['h4'])
    devRELUh4 = RELUDerivative(layer4)
    
    temp = np.matmul(weightsFinal["h4"], np.multiply(devRELUh4.T, weightsFinal["h4Out"]))
    temp = np.matmul(weightsFinal["h3"], np.multiply(devRELUh3.T, temp))
    temp = np.matmul(weightsFinal["h2"], np.multiply(devRELUh2.T, temp))
    inputEffects = np.matmul(weightsFinal["h1"], np.multiply(devRELUh1.T, temp))
    return inputEffects

def quintLayerInputEffects(x, weightsFinal):
    x = np.array([x.astype("float32")])
    layer1 = np.matmul(x, weightsFinal['h1'])
    devRELUh1 = RELUDerivative(layer1)
    
    layer1 = RELU(layer1)
    layer2 = np.matmul(layer1, weightsFinal['h2'])
    devRELUh2 = RELUDerivative(layer2)
    
    layer2 = RELU(layer2)
    layer3 = np.matmul(layer2, weightsFinal['h3'])
    devRELUh3 = RELUDerivative(layer3)

    layer3 = RELU(layer3)
    layer4 = np.matmul(layer3, weightsFinal['h4'])
    devRELUh4 = RELUDerivative(layer4)
    
    layer4 = RELU(layer4)
    layer5 = np.matmul(layer4, weightsFinal['h5'])
    print(layer5)
    devRELUh5 = RELUDerivative(layer5)
    
    temp = np.matmul(weightsFinal["h5"], np.multiply(devRELUh5.T, weightsFinal["h5Out"]))
    temp = np.matmul(weightsFinal["h4"], np.multiply(devRELUh4.T, temp))
    temp = np.matmul(weightsFinal["h3"], np.multiply(devRELUh3.T, temp))
    temp = np.matmul(weightsFinal["h2"], np.multiply(devRELUh2.T, temp))
    inputEffects = np.matmul(weightsFinal["h1"], np.multiply(devRELUh1.T, temp))
    return inputEffects

#Into nTiles
def financialsIntoTiles(data, n):
    for column in list(x_train.columns):
        data[column] = data[column].fillna(0)
        data[column] = pd.qcut(data[column],n,labels=False, duplicates = 'drop')
    return data

#Cost Functions
def MSElossFn(y, preds):
    squaredDifference = tf.square(y - preds)
    return tf.reduce_mean(squaredDifference, axis=-1)

#Activation Functions
def RELU(layer):
    result = np.zeros(layer.shape)
    for i in range(len(layer[0,])):
        result[0,i] = max(0, layer[0,i])
    return result

def RELUDerivative(layer):
    result = np.zeros(layer.shape)
    for i in range(len(layer[0,])):
        if layer[0,i] > 0:
            result[0,i] = 1
        else:
            result[0,i] = 0
    return result

#Average Input Output Effects
def averageInputEffect(x_train, weightsFinal, nLayerInputEffects):
    result = np.zeros((28, 1))
    for i in range(len(x_train)):
        result += (nLayerInputEffects(x_train.iloc[i], weightsFinal)/len(x_train))
    return result

def bullMarketData(data):
    #https://www.cnbc.com/2020/03/14/a-look-at-bear-and-bull-markets-through-history.html
    startDates = ["1932/06/01", "1942/04/22", "1949/06/14", "1957/10/22", "1962/06/27", "1966/10/07", "1970/05/26", "1974/10/3", "1982/08/12", "1987/12/04", "1990/10/11", "2002/10/09", "2009/03/09"]
    endDates = ["1937/03/05", "1946/05/28", "1956/08/01", "1961/12/11", "1966/02/08", "1968-11-28", "1973/01/10", "1980/11/27", "1987/08/24", "1990/07/15", "2000/03/23", "2007/10/08", "2020/02/18"]
    data = data[((data["date"] > startDates[5]) & (data["date"] < endDates[5])) | ((data["date"] > startDates[6]) & (data["date"] < endDates[6]))| ((data["date"] > startDates[7]) & (data["date"] < endDates[7]))| ((data["date"] > startDates[8]) & (data["date"] < endDates[8]))| ((data["date"] > startDates[9]) & (data["date"] < endDates[9]))| ((data["date"] > startDates[10]) & (data["date"] < endDates[10]))| ((data["date"] > startDates[11]) & (data["date"] < endDates[11]))| ((data["date"] > startDates[12]) & (data["date"] < endDates[12]))]
    return data

def bearMarketData(data):
    #https://www.cnbc.com/2020/03/14/a-look-at-bear-and-bull-markets-through-history.html
    startDates = ["1937/03/06", "1946/05/29", "1956/08/02", "1961/12/12", "1966/02/09", "1968/11/29", "1973/01/11", "1980/11/28", "1987/08/25", "1990/07/16", "2000/03/24", "2007/10/09", "2020/02/19"]
    endDates = ["1942/04/28","1949/06/13", "1957/10/21", "1962/06/26", "1966/10/06", "1970/05/25", "1974/10/2", "1982/08/11", "1987/12/03", "1990/10/10", "2002/10/08", "2009/03/08", "2020/04/21"]
    data = data[((data["date"] > startDates[5]) & (data["date"] < endDates[5])) | ((data["date"] > startDates[6]) & (data["date"] < endDates[6]))| ((data["date"] > startDates[7]) & (data["date"] < endDates[7]))| ((data["date"] > startDates[8]) & (data["date"] < endDates[8]))| ((data["date"] > startDates[9]) & (data["date"] < endDates[9]))| ((data["date"] > startDates[10]) & (data["date"] < endDates[10]))| ((data["date"] > startDates[11]) & (data["date"] < endDates[11]))| ((data["date"] > startDates[12]) & (data["date"] < endDates[12]))]
    return data 

In [153]:
#Predictions
singleLayerPreds = singleLayer(x, weights, biases)
doubleLayerPreds = doubleLayer(x, weights, biases)
tripleLayerPreds = tripleLayer(x, weights, biases)
quadLayerPreds = QuadLayer(x, weights, biases)
quintLayerPreds = QuintLayer(x, weights, biases)

trainingEpochs = 1
learningRate = 0.001
#Used to see the trajetory of cost in NN
displayFrequency = 100
#Batch Size 1 so SGD
batchSize = 1


#cost = MSElossFn(y, doubleLayerPreds)
#cost = abs(y - doubleLayerPreds)
#cost = tf.keras.losses.MeanSquaredLogarithmicError(y, doubleLayerPreds)
#cost = tf.keras.losses.logcosh(y, doubleLayerPreds)
h = tf.keras.losses.Huber(delta=1.0)
cost = h(y, doubleLayerPreds)
optimizer = tf.train.AdamOptimizer(learningRate).minimize(cost)

In [154]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(trainingEpochs):
        avg_cost = 0.0
        totalBatch = int(len(x_train) / batchSize)
        xBatchs = np.array_split(x_train, totalBatch)
        yBatchs = np.array_split(y_train, totalBatch)
        for i in range(totalBatch):
            xBatch, yBatch = xBatchs[i], yBatchs[i]
            _, c = sess.run([optimizer, cost], feed_dict = {x: xBatch, y: yBatch})
            avg_cost += c / totalBatch
        #if epoch % display_step == 0:
            #print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    weightsFinal = {
    'h1': weights["h1"].eval(),
    'h1Out': weights["h1Out"].eval(), 'h2': weights["h2"].eval(),
    'h2Out': weights["h2Out"].eval(), 'h3': weights["h3"].eval(),
    'h3Out': weights["h3Out"].eval(), 'h4': weights["h4"].eval(),
    'h4Out': weights["h4Out"].eval(), 'h5': weights["h5"].eval(),
    'h5Out': weights["h5Out"].eval()}
    biasesFinal = {
    'b1': biases["b1"].eval(),'b2': biases["b2"].eval(), 'b3': biases["b3"].eval(), 'b4': biases["b4"].eval(), 'b5': biases["b5"].eval(), 'bOut': biases["b5"].eval()}
    
    print("Finished!")

Finished!


In [155]:
#singleLayerAverageInputEffect = pd.DataFrame(averageInputEffect(x_train, weightsFinal, singleLayerInputEffects).T, columns = x_train.columns)
doubleLayerAverageInputEffect = pd.DataFrame(averageInputEffect(x_train, weightsFinal, doubleLayerInputEffects).T, columns = x_train.columns)
#tripleLayerAverageInputEffect = pd.DataFrame(averageInputEffect(x_train, weightsFinal, tripleLayerInputEffects).T, columns = x_train.columns)
#quadLayerAverageInputEffect = pd.DataFrame(averageInputEffect(x_train, weightsFinal, quadLayerInputEffects).T, columns = x_train.columns)
#quintLayerAverageInputEffect = pd.DataFrame(averageInputEffect(x_train, weightsFinal, quintLayerInputEffects).T, columns = x_train.columns)

In [156]:
pd.set_option('display.max_columns', None)
####NEW tests###
#0a
##epochs = 10, single layer, sample size = first 10000, learning rate = 0.001, cost function = MSE, NaN to 0
#Factors: Momentum, VOL, ptb, ps, -debt_invcap, aftret_equity, roce, 
#Importance: 0.273931, 0.271067, 0.191415, 0.123576, -0.096141, 0.08438, 0.083456
#test0a = lastMonth[(lastMonth["VOL"] >= 7) & (lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["aftret_equity"] >= 5) & (lastMonth["debt_invcap"] <= 5) & (lastMonth["roce"] >= 5)]
#Bull Results: SharpeRatioPortfolio 0.2344973414274156 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio 0.19496231388075952 SharpeRatioSP500 0.16197363811499685

#0b
##epochs = 10, single layer, sample size = first 10000, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: VOL, Momentum, ptb, ps, aftret_equity, -debt_invcap
#Importance: 0.336793, 0.318895, 0.171109, 0.127295, 0.095728, -0.088226
#test0b = lastMonth[(lastMonth["VOL"] >= 7) & (lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["aftret_equity"] >= 5) & (lastMonth["debt_invcap"] <= 5)]
#Bull Results: SharpeRatioPortfolio 0.25706943421881306 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio 0.25091871432046936 SharpeRatioSP500 0.16197363811499685

#0c
#Factors: Momentum
#test0c = lastMonth[(lastMonth["Momentum"] >= 7)]
#Bull Results: SharpeRatioPortfolio 0.22404764304379268 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio 0.19343144613202887 SharpeRatioSP500 0.16197363811499685

#1a
#epochs = 1, single layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, quick_ratio, -dltt_be
#Importance: 0.374932,  0.25162, 0.174929, -0.088003, 0.071433, -0.069968
#test1a = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["quick_ratio"] >= 5) & (lastMonth["dltt_be"] <= 5)]
#Bull Results: SharpeRatioPortfolio 0.18739101108073405 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio -0.14600395255388693 SharpeRatioSP500 0.16197363811499685

#1b
#epochs = 1, double layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, -dltt_be, roce
#Importance: 0.406814,  0.248615, 0.168675, -0.094921, -0.075894, 0.067618
#test1b = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["dltt_be"] <= 5) & (lastMonth["roce"] >= 5)]
#Bull Results: SharpeRatioPortfolio 0.22429904832471698 SharpeRatioSP500 0.16344536811675658
#Bear Results: SSharpeRatioPortfolio -0.04871466623141356 SharpeRatioSP500 0.16197363811499685

#1c
#epochs = 1, triple layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, -dltt_be, roce, quick_ratio
#Importance: 0.41733,  0.244493, 0.164006, -0.088569, -0.070572, 0.067626, 0.066897
#test1c = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["dltt_be"] <= 5) & (lastMonth["roce"] >= 5) & (lastMonth["quick_ratio"] >= 5)]
#Bull Results: SharpeRatioPortfolio 0.2177175358668748 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio 0.3042474774204732 SharpeRatioSP500 0.16197363811499685

#1d
#epochs = 1, quad layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, -dltt_be, roce, quick_ratio
#Importance: 0.409525,  0.261579, 0.186323, -0.084871, -0.067847, 0.066811, 0.066294
#test1d = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["dltt_be"] <= 5) & (lastMonth["roce"] >= 5) & (lastMonth["quick_ratio"] >= 5)]
#Bull Results: SharpeRatioPortfolio 0.2177175358668748 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio 0.3042474774204732 SharpeRatioSP500 0.16197363811499685

#1e
#epochs = 1, quint layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: None
#Importance: None
#Bull Results: None
#Bear Results: None

#2a
#epochs = 1, double layer, sample size = one month of each year, learning rate = 0.001, cost function = MSA, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, quick_ratio, roce
#Importance: 0.466551, 0.321029, 0.189773, -0.103481, 0.097529, 0.092457
#test2a = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 6) &  (lastMonth["quick_ratio"] >= 6) & (lastMonth["roce"] >= 6)]
#Bull Results: SharpeRatioPortfolio 0.23200614283467674 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio 0.27738015389011944 SharpeRatioSP500 0.16197363811499685

#2b
#epochs = 1, double layer, sample size = one month of each year, learning rate = 0.001, cost function = MSLE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, -dltt_be, quick_ratio
#Importance: 0.427268,  0.269829, 0.184234, -0.100088, -0.079814, 0.074181
#test2b = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 6) &  (lastMonth["dltt_be"] <= 5) & (lastMonth["quick_ratio"] >= 5) ]
#Bull Results: SharpeRatioPortfolio 0.22476676476539456 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio 0.27498890485154603 SharpeRatioSP500 0.16197363811499685

#2c
#epochs = 1, double layer, sample size = one month of each year, learning rate = 0.001, cost function = Logcosh, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, roce, dltt_be
#Importance: 0.395645,  0.262563, 0.172883, -0.092535, 0.073913, -0.073714
#test2c = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 6) &  (lastMonth["roce"] >= 5) & (lastMonth["dltt_be"] <= 5) ]
#Bull Results: SharpeRatioPortfolio 0.2648859071735201 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio -0.1567685540261145 SharpeRatioSP500 0.16197363811499685

#2d
#epochs = 1, double layer, sample size = one month of each year, learning rate = 0.001, cost function = huber, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, -dltt_be, roce
#Importance: 0.385051,  0.260463, 0.174529, -0.089277, -0.07442, 0.072428
#test2d = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) &  (lastMonth["dltt_be"] <= 5) & (lastMonth["roce"] >= 5) ]
#Bull Results: SharpeRatioPortfolio 0.2759390194303892 SharpeRatioSP500 0.16344536811675658
#Bear Results: SharpeRatioPortfolio -0.04871466623141356 SharpeRatioSP500 0.16197363811499685


,VOL,Momentum,CAPEI,bm,evm,pe_inc,ps,pcf,ptpm,roa,roe,roce,aftret_invcapx,aftret_equity,GProf,equity_invcap,debt_invcap,capital_ratio,invt_act,rect_act,cash_debt,dltt_be,de_ratio,cash_ratio,quick_ratio,curr_ratio,rd_sale,ptb
0,-0.032642,0.385051,-0.010932,-0.051442,0.017192,-0.025635,0.260463,-0.023886,0.031694,0.059669,0.00331,0.072428,-0.036083,-0.013598,0.011206,0.036465,0.049032,0.038148,0.016381,0.038775,0.027834,-0.07442,0.025544,-0.089277,0.064109,0.044675,0.024273,0.174529


In [51]:
bearData = pd.read_csv("bearData.csv")
bearData["date"] = bearData.date.str.replace("-", "/")
bearData = bearMarketData(bearData)
bearData["date"] = bearData.date.str.replace("/", "-")
bearData["Month"] = bearData["date"].str[:7]
bearData = bearData[["TICKER", "PRC", "date", "Month"]]

trainBullData = pd.read_csv("trainBullDataNorm.csv")
lastMonth = trainBullData[trainBullData["Month"] == "2007/09"]
lastMonth = financialsIntoTiles(lastMonth, 10)

testBullData = pd.read_csv("testBullDataNorm.csv")
testBullData["date"] = testBullData.date.str.replace("/", "-")
testBullData["Month"] = testBullData["date"].str[:7]
testBullData = testBullData[["TICKER", "PRC", "date", "Month"]]

sp500 = pd.read_csv("^GSPC.csv")
sp500["date"] = sp500["Date"]
sp500BearData = bearMarketData(sp500)
sp500BearData["date"] = sp500BearData.date.str.replace("/", "-")
sp500BearData["Month"] = sp500BearData["date"].str[:7]
sp500BearData = sp500BearData[["Month", "Adj Close", "date"]]
sp500BullData = bullMarketData(sp500)
sp500BullData["date"] = sp500BullData.date.str.replace("/", "-")
sp500BullData["Month"] = sp500BullData["date"].str[:7]
sp500BullData = sp500BullData[["Month", "Adj Close", "date"]]

bearMonthIntersection = list(set(list(sp500BearData.Month.unique())).intersection(list(bearData.Month.unique())))
bearData = bearData[bearData["Month"].isin(bearMonthIntersection)]
sp500BearData = sp500BearData[sp500BearData["Month"].isin(bearMonthIntersection)]

bullMonthIntersection = list(set(list(sp500BullData.Month.unique())).intersection(list(testBullData.Month.unique())))
testBullData = testBullData[testBullData["Month"].isin(bullMonthIntersection)]
sp500BullData = sp500BullData[sp500BullData["Month"].isin(bullMonthIntersection)]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

In [157]:
test0a = lastMonth[(lastMonth["VOL"] >= 7) & (lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["aftret_equity"] >= 5) & (lastMonth["debt_invcap"] <= 5) & (lastMonth["roce"] >= 5)]
test0b = lastMonth[(lastMonth["VOL"] >= 7) & (lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["aftret_equity"] >= 5) & (lastMonth["debt_invcap"] <= 5)]
test0c = lastMonth[(lastMonth["Momentum"] >= 7)]
test1a = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["quick_ratio"] >= 5) & (lastMonth["dltt_be"] <= 5)]
test1b = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["dltt_be"] <= 5) & (lastMonth["roce"] >= 5)]
test1c = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["dltt_be"] <= 5) & (lastMonth["roce"] >= 5) & (lastMonth["quick_ratio"] >= 5)]
test1d = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["dltt_be"] <= 5) & (lastMonth["roce"] >= 5) & (lastMonth["quick_ratio"] >= 5)]
test2a = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 6) &  (lastMonth["quick_ratio"] >= 6) & (lastMonth["roce"] >= 6)]
test2b = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 6) &  (lastMonth["dltt_be"] <= 5) & (lastMonth["quick_ratio"] >= 5) ]
test2c = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 6) &  (lastMonth["roce"] >= 5) & (lastMonth["dltt_be"] <= 5) ]
test2d = lastMonth[(lastMonth["Momentum"] >= 7) & (lastMonth["ps"] >= 7) & (lastMonth["ptb"] >= 7) & (lastMonth["cash_ratio"] <= 5) &  (lastMonth["dltt_be"] <= 5) & (lastMonth["roce"] >= 5) ]

testPortfolio = test2d

In [158]:
#Finding Bull Market Sharpe Ratios
portfolio = pd.DataFrame(testBullData.Month.unique()[1:], columns = ["Month"])
for stock in list(testPortfolio["TICKER"]):
    newDF = testBullData[testBullData["TICKER"] == stock].iloc[1:]
    if newDF.shape[0]  == portfolio.shape[0]:
        normReturn = list(newDF["PRC"] / newDF.iloc[0]["PRC"])
        portfolio[stock] = normReturn
        
portfolio = portfolio.dropna(axis=1, how='any')
portfolio["Total"] = portfolio.sum(axis=1)
portfolio["Return"] = portfolio["Total"].pct_change(1)
sp500["Return"] = sp500["Adj Close"].pct_change(1)

SharpeRatioPortfolio = portfolio["Return"].mean() / portfolio["Return"].std()
SharpeRatioSP500 = sp500["Return"].mean() / sp500["Return"].std()
print("SharpeRatioPortfolio", SharpeRatioPortfolio, "SharpeRatioSP500", SharpeRatioSP500)

SharpeRatioPortfolio 0.2759390194303892 SharpeRatioSP500 0.16344536811675658


In [159]:
#Finding Bear Market Sharpe Ratios
portfolioList = testPortfolio.TICKER.unique()
portfolio = bearData[bearData.TICKER.isin(portfolioList)]
startDate = portfolio["date"].min()
endDate = portfolio["date"].max()
mux = pd.MultiIndex.from_product([pd.date_range(startDate, endDate, freq='m'), portfolio['TICKER'].unique()], names=['date','TICKER'])
result = portfolio.set_index(['date','TICKER']).reindex(mux, fill_value= -10000).reset_index()
result['date']= result['date'].astype(str)
result['TICKER']= result['TICKER'].astype(str)
result = pd.merge(result, portfolio, how='left', on=['date', 'TICKER'])
result.fillna(0, inplace=True)
result["PRC"] = result[['PRC_x', 'PRC_y']].values.max(1)
result["Month"] = result["date"].str[:7]
result = result[["Month", "TICKER", "PRC"]]
result = result[result["Month"].isin(bearMonthIntersection)]

portfolio = pd.DataFrame(result.Month.unique(), columns = ["Month"])
result = result.sort_values(by = ["TICKER", "Month"], axis = 0)
for stock in result.TICKER.unique():
    newDF = result[result["TICKER"] == stock]
    if newDF.shape[0]  == portfolio.shape[0]:
        normReturn = list(newDF["PRC"])
        portfolio[stock] = normReturn

portfolio["Total"] = portfolio.sum(axis=1)
portfolio["SP_PRC"] = list(sp500BearData["Adj Close"])[:-1]
portfolio = portfolio.loc[portfolio['Total'] != 0]
portfolio["Return"] = portfolio["Total"].pct_change(1)
portfolio["SP_Return"] = portfolio["SP_PRC"].pct_change(1)
SharpeRatioPortfolio = portfolio["Return"].mean() / portfolio["Return"].std()
SharpeRatioSP500 = portfolio["SP_Return"].mean() / portfolio["SP_Return"].std()
print("SharpeRatioPortfolio", SharpeRatioPortfolio, "SharpeRatioSP500", SharpeRatioSP500)


SharpeRatioPortfolio -0.04871466623141356 SharpeRatioSP500 0.16197363811499685


In [2]:
Extra Code

In [ ]:
test1 = lastMonth[(lastMonth["VOL"] >= 5) & (lastMonth["Momentum"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["roce"] >= 5) & (lastMonth["aftret_equity"] >= 5)]
test2 = lastMonth[(lastMonth["VOL"] >= 5) & (lastMonth["Momentum"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["debt_invcap"] <= 5) & (lastMonth["curr_ratio"] <= 5)]
test3 = lastMonth[(lastMonth["Momentum"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["capital_ratio"] >= 5) & (lastMonth["dltt_be"] <= 5)]
test4 = lastMonth[(lastMonth["Momentum"] >= 5)]
test5 = lastMonth[(lastMonth["Momentum"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["dltt_be"] <= 5)]
test6 = lastMonth[(lastMonth["Momentum"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["quick_ratio"] >= 5)]
test7 = lastMonth[(lastMonth["Momentum"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["roce"] >= 5)]
test8 = lastMonth[(lastMonth["Momentum"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["roce"] >= 5) & (lastMonth["dltt_be"] <= 5)]
test10 = lastMonth[(lastMonth["VOL"] >= 5) & (lastMonth["Momentum"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["roce"] >= 5) & (lastMonth["aftret_equity"] >= 5)]
test11 = lastMonth[(lastMonth["Momentum"] >= 5) & (lastMonth["ps"] >= 5) & (lastMonth["ptb"] >= 5) & (lastMonth["cash_ratio"] <= 5) & (lastMonth["roce"] >= 5) ]

###Top Input Factors by Abs###

####1####
##epochs = 10, single layer, sample size = 10000, learning rate = 0.001, cost function = MSE, NaN to 0
#Factors: VOL, Momentum, ptb, ps, roce, aftret_equity
#Results: SharpeRatioPortfolio 0.28691530240442353

####2####
##epochs = 10, single layer, sample size = first 10000, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, VOL, ptb, ps, -debt_invcap, -curr_ratio, 
#Results: SharpeRatioPortfolio 0.2958103386673036 SharpeRatioSP500 0.285965412597795

####3####
##epochs = 10, single layer, sample size =last 10000, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -dltt_be, capital_ratio
#Results: SharpeRatioPortfolio 0.16170813087034738 SharpeRatioSP500 0.285965412597795

####4####
#Factors: Momentum
#SharpeRatioPortfolio 0.2585126296237325

####5####
##epochs = 1, single layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, dltt_be
#Importance: 0.35515,  0.253058, 0.186872, -0.087725, -0.080792
#Results: SharpeRatioPortfolio 0.2742636470407426 SharpeRatioSP500 0.285965412597795

####6####
##epochs = 1, double layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, quick_ratio
#Importance: 0.41848,  0.176023, 0.186872, -0.091075, 0.071822
#Results: SharpeRatioPortfolio 0.2631329253582494 SharpeRatioSP500 0.285965412597795

####7####
##epochs = 1, triple layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, roce
#Importance: 0.39964,  0.265535, 0.191515, -0.089483, 0.069108
#Results: SharpeRatioPortfolio 0.18507377730816127 SharpeRatioSP500 0.285965412597795

####8####
##epochs = 1, quad layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, roce, -dltt_be
#Importance: 0.406719,  0.246568, 0.173938, -0.083529, 0.070508, -0.06617
#Results: SharpeRatioPortfolio 0.2780929633030081 SharpeRatioSP500 0.285965412597795


####9####
##epochs = 1, quint layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 1
#Factors: 

####10####
##epochs = 1, triple layer, sample size = one month of each year, learning rate = 0.001, cost function = MSE, NaN to 0
#Factors: Momentum, ps, ptb, quick_ratio, -rd_sale, 
#Importance: 0.393965,  0.283151, 0.173343, 0.10103, -0.08092
#Results: SharpeRatioPortfolio 0.2780929633030081 SharpeRatioSP500 0.285965412597795

####11####
##epochs = 1, triple layer, sample size = one month of each year, learning rate = 0.001, cost function = MSA, NaN to 1
#Factors: Momentum, ps, ptb, -cash_ratio, roce
#Importance: 0.426547,  0.248287, 0.165419, -0.084407, 0.069641
#Results: SharpeRatioPortfolio 0.2780929633030081 SharpeRatioSP500 0.285965412597795




In [ ]:
#Finding Bull Market Sharpe Ratios
portfolioList = testPortfolio.TICKER.unique()
portfolio = testBullData[testBullData.TICKER.isin(portfolioList)]
startDate = portfolio["date"].min()
endDate = portfolio["date"].max()
mux = pd.MultiIndex.from_product([pd.date_range(startDate, endDate, freq='m'), portfolio['TICKER'].unique()], names=['date','TICKER'])
result = portfolio.set_index(['date','TICKER']).reindex(mux, fill_value= -10000).reset_index()
result['date']= result['date'].astype(str)
result['TICKER']= result['TICKER'].astype(str)

result = pd.merge(result, portfolio, how='left', on=['date', 'TICKER'])
result.fillna(0, inplace=True)
result["PRC"] = result[['PRC_x', 'PRC_y']].values.max(1)
result["Month"] = result["date"].str[:7]
result = result[["Month", "TICKER", "PRC"]]
result = result[result["Month"].isin(bullMonthIntersection)]

portfolio = pd.DataFrame(result.Month.unique(), columns = ["Month"])
result = result.sort_values(by = ["TICKER", "Month"], axis = 0)
for stock in result.TICKER.unique():
    newDF = result[result["TICKER"] == stock]
    if newDF.shape[0]  == portfolio.shape[0]:
        normReturn = list(newDF["PRC"])
        portfolio[stock] = normReturn

portfolio["Total"] = portfolio.sum(axis=1)
portfolio["SP_PRC"] = list(sp500BullData["Adj Close"])
portfolio = portfolio.loc[portfolio['Total'] != 0]
portfolio["Return"] = portfolio["Total"].pct_change(1)
portfolio["SP_Return"] = portfolio["SP_PRC"].pct_change(1)
print(portfolio)
SharpeRatioPortfolio = portfolio["Return"].mean() / portfolio["Return"].std()
SharpeRatioSP500 = portfolio["SP_Return"].mean() / portfolio["SP_Return"].std()
print("SharpeRatioPortfolio", SharpeRatioPortfolio, "SharpeRatioSP500", SharpeRatioSP500)

In [98]:
trainBullData = pd.read_csv("trainBullDataNorm.csv")
testBullData = pd.read_csv("testBullDataNorm.csv")
trainBullData.fillna(0, inplace=True)
testBullData.fillna(0, inplace=True)
trainBullData = trainBullData.sort_values(by = ["PERMNO", "Month"])
trainBullData["Return"] = trainBullData["PRC"].pct_change(1)
trainBullData = trainBullData[["TICKER", "Return", "Month"]]
#for testing size
trainBullData.dropna(inplace=True)

In [57]:
stocks = trainBullData.TICKER.unique()
covMatrix = collections.defaultdict(float)
for num1 in range(len(stocks)):
    stock1 = trainBullData[trainBullData["TICKER"] == stocks[num1]]
    stock1[stocks[num1]] = stock1["Return"]
    for num2 in range(num1 + 1,len(stocks)):
        stock2 = trainBullData[trainBullData["TICKER"] == stocks[num2]]
        stock2[stocks[num2]] = stock2["Return"]
        merged = pd.merge(stock1, stock2, on= "Month", how= "inner")
        merged = merged[[stocks[num1], stocks[num2]]]
        covMatrix[(num1, num2)] = merged.cov()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2449: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:390: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis

KeyboardInterrupt: 

In [78]:
trainBullData.replace([np.inf, np.inf], 0, inplace = True)
stocks = trainBullData.TICKER.unique()
std = {i: trainBullData[trainBullData["TICKER"] == stocks[i]]["Return"].std(axis = 0) for i in range(len(stocks))}

In [99]:
sums = {i: trainBullData[trainBullData["TICKER"] == stocks[i]]["Return"].sum(axis = 0) for i in range(len(stocks))}

In [84]:
trainBullData = trainBullData[trainBullData["Month"].str.contains("05")]
stocks = trainBullData.TICKER.unique()
x_train = np.ones([64])

x = tf.placeholder("float", shape= [None, len(x_train)])
y = tf.placeholder("float", shape= [None, len(stocks)])

weights = {
    'h1': tf.Variable(tf.random_normal([len(x_train), 32], stddev=1/28)),
    'h1Out': tf.Variable(tf.random_normal([32, len(stocks)], stddev=1/32)), 'h2': tf.Variable(tf.random_normal([32, 16], stddev=1/32)),
    'h2Out': tf.Variable(tf.random_normal([16, len(stocks)], stddev=1/16)), 'h3': tf.Variable(tf.random_normal([16, 8], stddev=1/16)),
    'h3Out': tf.Variable(tf.random_normal([8, len(stocks)], stddev=1/8)), 'h4': tf.Variable(tf.random_normal([8, 4], stddev=1/8)),
    'h4Out': tf.Variable(tf.random_normal([4, len(stocks)], stddev=1/4)), 'h5': tf.Variable(tf.random_normal([4, 2], stddev=1/4)),
    'h5Out': tf.Variable(tf.random_normal([2, len(stocks)], stddev=1/2))}


In [100]:
def singleLayer(x, weights):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layerOut = tf.matmul(layer1, weights['h1Out'])
    return layerOut
    
def doubleLayer(x, weights):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.matmul(layer1, weights['h2'])
    layer2 = tf.nn.relu(layer2)
    layerOut = tf.matmul(layer2, weights['h2Out'])
    return layerOut
    
def tripleLayer(x, weights):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.matmul(layer1, weights['h2'])
    layer2 = tf.nn.relu(layer2)
    layer3 = tf.matmul(layer2, weights['h3'])
    layer3 = tf.nn.relu(layer3)
    layerOut = tf.matmul(layer3, weights['h3Out'])
    return layerOut

def QuadLayer(x, weights):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.matmul(layer1, weights['h2'])
    layer2 = tf.nn.relu(layer2)
    layer3 = tf.matmul(layer2, weights['h3'])
    layer3 = tf.nn.relu(layer3)
    layer4 = tf.matmul(layer3, weights['h4'])
    layer4 = tf.nn.relu(layer4)
    layerOut = tf.matmul(layer4, weights['h4Out'])
    return layerOut

def QuintLayer(x, weights):
    layer1 = tf.matmul(x, weights['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.matmul(layer1, weights['h2'])
    layer2 = tf.nn.relu(layer2)
    layer3 = tf.matmul(layer2, weights['h3'])
    layer3 = tf.nn.relu(layer3)
    layer4 = tf.matmul(layer3, weights['h4'])
    layer4 = tf.nn.relu(layer4)
    layer5 = tf.matmul(layer4, weights['h5'])
    layer5 = tf.nn.relu(layer5)
    layerOut = tf.matmul(layer5, weights['h5Out'])
    return layerOut

def SharpeRatioCost(y, sums, std, trainBullData):
    sharpeRatio = 0
    divisor = 0
    for num1 in range(len(stocks)):
        sharpeRatio += y[num1]* sums[num1]
        for num2 in range(num1, len(stocks)):
            if num1 != num2:
                divisor += 2 * y[num1] * y[num2] * std[num1] * std[num2]
            else:
                divisor += y[num1]**2 * std[num1]**2
    sharpeRatio = sharpeRatio/ (divisor**(1/2))
    return sharpeRatio
    
        

In [101]:
#Predictions
singleLayerPreds = singleLayer(x, weights)
doubleLayerPreds = doubleLayer(x, weights)
tripleLayerPreds = tripleLayer(x, weights)
quadLayerPreds = QuadLayer(x, weights)
quintLayerPreds = QuintLayer(x, weights)

trainingEpochs = 10
learningRate = 0.001
#Batch Size 1 so SGD
batchSize = 1


cost = SharpeRatioCost(singleLayerPreds, sums, std, trainBullData)
optimizer = tf.train.AdamOptimizer(learningRate).minimize(cost)

KeyboardInterrupt: 

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(trainingEpochs):
        avg_cost = 0.0
        totalBatch = int(len(x_train) / batchSize)
        xBatchs = np.array_split(x_train, totalBatch)
        for i in range(totalBatch):
            xBatch, yBatch = xBatchs[i], yBatchs[i]
            _, c = sess.run([optimizer, cost], feed_dict = {x: xBatch})
            avg_cost += c / totalBatch
        #if epoch % display_step == 0:
            #print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Finished!")